<a href="https://colab.research.google.com/github/MariaElisaOliveiraMartins/DS_Analises/blob/main/etl_pns2019_tot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descrição deste trabalho: 

Objetivo: elaboração de um processo de ETL da base de dados PNS de 2019 para efeito de estudo independente sobre as condições de saúde da população idosa. 

Fonte de Dados: Site do IBGE

Processo:

1) Extração aleatória de uma amostra da base PNS.

2) Carga do arquivo de metadados da base PNS.

3) Geração de um novo dataframe preenchido com os dados da amostra nas respectivas colunas, de acordo com os metadados. 

4) Limpeza e gravação do dataframe gerado.

Resultado: geração de um arquivo cvs contendo uma amostra já tratada dos dados extraídos da base PNS. 

Data do trabalho: Dez/2020

In [1]:
# Acessa o Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importa as bibliotecas

import pandas as pd 
import numpy as np 
import seaborn as sbn 
from matplotlib import pyplot as plt

In [3]:
# Carrega a base de dados do PNS.

base_PNS = pd.read_csv('/content/drive/MyDrive/Dados_DS/PNS_2019.txt') 
#bkp_base_PNS = base_PNS.copy()
base_PNS.shape 

(293725, 1)

In [13]:
# Extrai aleatóriamente uma amostra da base PNS e ordena pela nro. da linha.

amostra = base_PNS.sample(frac=0.01, replace=True) # pega 1%
df_amostra_PNS = pd.DataFrame(amostra).sort_index()
df_amostra_PNS.shape

(2937, 1)

In [15]:
# Confere a amostra obtida (início).

df_amostra_PNS.head()

,REGS
6,111110011110000016000201201904110 111104033215...
146,111110011110000584000401201905119 111105021 15...
159,1111100111100005840008052019 .11 . . ...
271,111110011110002149001201201905110 111108033215...
337,111110011110002929000201201907111 141308043215...


In [16]:
# Confere a amostra obtida (final).

df_amostra_PNS.tail()

,REGS
293393,535310220530023953000101201902210 111107011 11...
293467,535310220530025265001201201904211 111305022216...
293487,535310220530028975000401201904210 111106033211...
293521,535310220530028975001501201902211 111307013211...
293710,535310220530044434001001201902210 111305022216...


In [17]:
# Confere acesso aos dados da amostra pelo indice do dataframe. 

print('primeira linha da amostra = ' , df_amostra_PNS.iloc[0])
print('última linha da amostra   = ' , df_amostra_PNS.iloc[df_amostra_PNS.shape[0]-1])

primeira linha da amostra =  REGS    111110011110000016000201201904110 111104033215...
Name: 6, dtype: object
última linha da amostra   =  REGS    535310220530044434001001201902210 111305022216...
Name: 293710, dtype: object


In [20]:
# Carga da planilha de metadados da base PNS.

df_dic_PNS2019 = pd.read_csv('/content/drive/MyDrive/Dados_DS/dicionario_tot_PNS_microdados_2019.csv', sep='\t')
df_dic_PNS2019.shape

(682, 3)

In [21]:
# Confere a carga da planilha de metadados.

df_dic_PNS2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   offset   682 non-null    int64 
 1   collen   682 non-null    int64 
 2   colname  682 non-null    object
dtypes: int64(2), object(1)
memory usage: 16.1+ KB


In [22]:
df_dic_PNS2019.head()

,offset,collen,colname
0,1,2,V0001
1,3,7,V0024
2,10,9,UPA_PNS
3,19,4,V0006_PNS
4,23,2,V0015


In [26]:
# Gera uma nova coluna de offset subtraindo 1 posição para compatibilidade com o python.
df_dic_PNS2019['new_offset'] = df_dic_PNS2019['offset'] - 1
df_dic_PNS2019.head() 

,offset,collen,colname,new_offset
0,1,2,V0001,0
1,3,7,V0024,2
2,10,9,UPA_PNS,9
3,19,4,V0006_PNS,18
4,23,2,V0015,22


In [27]:
# Cria um novo dataframe para conter os dados que serão tratados e utilizados na análise. 

df_new_PNS2019 = pd.DataFrame(index=range(0, df_amostra_PNS.shape[0]), columns=df_dic_PNS2019['colname'])
df_new_PNS2019.shape

(2937, 682)

In [28]:
# Confere a geração do novo dataframe.

df_new_PNS2019.head()

colname,V0001,V0024,UPA_PNS,V0006_PNS,V0015,V0020,V0022,V0026,V0031,V0025A,V0025B,A001,A002010,A003010,A004010,A01001,A011,A005010,A005012,A00601,A009010,A01401,A01402,A01403,A01501,A016010,A018011,A018012,A018013,A018014,A018015,A018016,A018017,A018018,A018019,A018020,A018021,A018022,A018023,A018024,...,U005,U006,U00902,U01002,U014,U02001,U02101,U02302,U02303,U02402,U02403,U02501,W001,W00101,W00102,W00103,W00201,W00202,W00203,V0028,V0029,V0030,V00281,V00291,V00301,V00282,V00292,V00302,V00283,V00293,V00303,VDC001,VDC003,VDD004A,VDE001,VDE002,VDF002,VDF003,VDF004,VDDATA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# teste: seleção dos dados da amostra pelo offset.

aux = df_amostra_PNS.iloc[0]['REGS']
print(type(aux))
print(aux)
print(aux[0:10])

<class 'str'>
111110011110000016000201201904110 11110403321501 . 411011011012 .1031012 .2 .2 .12 .10100000001 .145504010121212198503312 .2   12  .4112  .  .  1071 101  2011        . . .152216   56011100000500        . 036        .        .   .  . .  .  . . .  222222211111112205612       .2       .2       .2       .122 .  .        1222 . .   .2 2 .2  .42 .    . .       .       012 .  . .    2     2           1                     .    .     .  .                                             .  .       . . .    . .   . .  . .   . .          .  . .  . .  . .  . .  . .  . .  . .   . . . . . .          .               .  .        .                             .  .   .           .                                         .             .        .         .             .         .        .          .  .       .           .           .                      .   .         .           .      .  .     .    .    .    .    .    . 00085.58908480     .                .        00118.31182322     .         

In [30]:
# Distribui e preenche o novo dataframe com os dados da amostra, conforme descreve os metadados.

qtdlin = df_new_PNS2019.shape[0]
qtdcol = df_new_PNS2019.shape[1]

linha = 0

for idx in range(0, qtdlin):
  
  aux = df_amostra_PNS.iloc[idx]['REGS']

  #print(linha, ' = ', idx, ' - ', aux, '\n')
  

  for ptr in  range(0, qtdcol):

    desloc = df_dic_PNS2019.iloc[ptr]['new_offset']
    tam = desloc + df_dic_PNS2019.iloc[ptr]['collen']
    coluna = df_dic_PNS2019.iloc[ptr]['colname']
    
    df_new_PNS2019.iloc[idx][coluna] = aux[desloc:tam]
    
    #print(desloc, tam, coluna, '|', aux[desloc:tam], '|')

  linha += 1

linha

2937

In [32]:
# Confere o resultado de preenchimento e distribuição dos dados da amostra no novo dataframe (inicio).

df_new_PNS2019.head()

colname,V0001,V0024,UPA_PNS,V0006_PNS,V0015,V0020,V0022,V0026,V0031,V0025A,V0025B,A001,A002010,A003010,A004010,A01001,A011,A005010,A005012,A00601,A009010,A01401,A01402,A01403,A01501,A016010,A018011,A018012,A018013,A018014,A018015,A018016,A018017,A018018,A018019,A018020,A018021,A018022,A018023,A018024,...,U005,U006,U00902,U01002,U014,U02001,U02101,U02302,U02303,U02402,U02403,U02501,W001,W00101,W00102,W00103,W00201,W00202,W00203,V0028,V0029,V0030,V00281,V00291,V00301,V00282,V00292,V00302,V00283,V00293,V00303,VDC001,VDC003,VDD004A,VDE001,VDE002,VDF002,VDF003,VDF004,VDDATA
0,11,1110011,110000016,0002,01,2019,04,1,1,0,,1,1,1,1,04,03,3,2,1,5,01,.,,4,1,1,01,1,01,1,01,2,.,1,03,1,01,2,.,...,,,.,,,,,,.,,.,,,.,.,.,.,.,.,00085.58908480,.,.,00118.31182322,.,.,000529479,.,.,111,.,.,04,04,2,1,1,00001000,00000250,2,20201105
1,11,1110011,110000584,0004,01,2019,05,1,1,9,,1,1,1,1,05,02,1,,1,5,01,.,,4,1,1,01,1,01,1,01,2,.,1,03,2,.,2,.,...,,,.,,,,,,.,,.,,,.,.,.,.,.,.,00072.78798283,.,.,00100.61655616,.,.,000529479,.,.,111,.,.,05,04,2,,,00002798,00000560,3,20201105
2,11,1110011,110000584,0008,05,2019,.,1,1,,,,,,,.,.,,,,,.,.,,,,,.,,.,,.,,.,,.,,.,,.,...,,,.,,,,,,.,,.,,,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,,,.,.,.,20201105
3,11,1110011,110002149,0012,01,2019,05,1,1,0,,1,1,1,1,08,03,3,2,1,5,02,.,,3,1,1,01,1,01,1,01,2,.,1,03,2,.,2,.,...,,,.,,,,,,.,,.,,,.,.,.,.,.,.,00115.12092075,.,.,00159.13438093,.,.,000529479,.,.,111,.,.,05,02,5,1,1,00003000,00000600,3,20201105
4,11,1110011,110002929,0002,01,2019,07,1,1,1,,1,4,1,3,08,04,3,2,1,5,01,.,,4,1,1,04,1,04,1,02,2,.,1,05,1,01,2,.,...,1,1,.,,,,,2,01,2,03,2,,.,.,.,.,.,.,00166.40917108,00832.04585538,.,00230.03134660,01412.16248282,.,000529479,00061277.87353323,.,111,11113,.,07,05,5,1,1,00009798,00001400,4,20201105


In [33]:
# Confere o resultado do processo preenchimento e distribuição dos dados da amostra no novo dataframe (final).

df_new_PNS2019.tail()

colname,V0001,V0024,UPA_PNS,V0006_PNS,V0015,V0020,V0022,V0026,V0031,V0025A,V0025B,A001,A002010,A003010,A004010,A01001,A011,A005010,A005012,A00601,A009010,A01401,A01402,A01403,A01501,A016010,A018011,A018012,A018013,A018014,A018015,A018016,A018017,A018018,A018019,A018020,A018021,A018022,A018023,A018024,...,U005,U006,U00902,U01002,U014,U02001,U02101,U02302,U02303,U02402,U02403,U02501,W001,W00101,W00102,W00103,W00201,W00202,W00203,V0028,V0029,V0030,V00281,V00291,V00301,V00282,V00292,V00302,V00283,V00293,V00303,VDC001,VDC003,VDD004A,VDE001,VDE002,VDF002,VDF003,VDF004,VDDATA
2932,53,5310220,530023953,0001,01,2019,02,2,1,0,,1,1,1,1,07,01,1,,1,1,01,.,,3,1,2,.,1,01,1,01,2,.,1,01,1,01,2,.,...,,,.,,,,,,.,,.,,,.,.,.,.,.,.,00176.93038493,.,.,00199.09440803,.,.,003015156,.,.,531,.,.,02,02,2,2,,00000998,00000499,2,20201105
2933,53,5310220,530025265,0012,01,2019,04,2,1,1,,1,1,1,3,05,02,2,2,1,6,01,.,,3,2,1,01,1,01,2,.,2,.,1,01,2,.,2,.,...,3,1,.,,,,,2,01,1,.,2,,.,.,.,.,.,.,00175.48214055,00350.96428111,.,00197.46474246,00392.07105408,.,003015156,00340544.33777183,.,531,53113,.,04,02,2,1,2,00000300,00000075,1,20201105
2934,53,5310220,530028975,0004,01,2019,04,2,1,0,,1,1,1,1,06,03,3,2,1,1,01,.,,4,2,1,01,1,01,1,02,2,.,1,03,1,01,1,01,...,,,.,,,,,,.,,.,,,.,.,.,.,.,.,00190.51169473,.,.,00214.37704496,.,.,003015156,.,.,531,.,.,04,03,5,1,1,00012999,00003250,6,20201105
2935,53,5310220,530028975,0015,01,2019,02,2,1,1,,1,1,1,3,07,01,3,2,1,1,02,.,,4,3,1,02,1,01,1,01,2,.,1,01,1,01,2,.,...,3,1,03,6,2,1,2,2,07,2,03,1,,.,.,.,.,.,.,00190.51169473,00381.02338945,.,00214.37704496,00421.56446507,.,003015156,00404146.28126234,.,531,53114,.,02,02,2,1,1,00002000,00001000,4,20201105
2936,53,5310220,530044434,0010,01,2019,02,2,1,0,,1,1,1,3,05,02,2,2,1,6,01,.,,3,1,1,01,1,01,1,01,2,.,1,02,1,01,1,02,...,,,.,,,,,,.,,.,,,.,.,.,.,.,.,00562.24598930,.,.,00632.67839752,.,.,003015156,.,.,531,.,.,02,02,4,1,1,00003000,00001500,4,20201105


In [38]:
# Verifica a existencias de nulos no novo dataframe.

(df_new_PNS2019[df_new_PNS2019.notna()].count()).values

array([2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937, 2937,
       2937, 2937, 2937, 2937, 2937, 2937, 2937, 29

In [ ]:
# Salva o resultado do processo de ETL no Google Drive

df_new_PNS2019.to_csv('/content/drive/MyDrive/Dados_DS/new_PNS2019.csv')

### Fim do Processo 
